In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import r2_score


In [2]:
np.random.seed(42)
x = np.random.randn(100, 3) 
y = 3*x[:,0] + 2*x[:,1] - 4*x[:,2] + np.random.randn(100)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

ridge=Ridge(alpha=1.0)
ridge.fit(x_train,y_train)
y_pred=ridge.predict(x_test)

print("Coefficients:", ridge.coef_)
print("Intercept:", ridge.intercept_)
print("r2 score:",r2_score(y_test,y_pred))

Coefficients: [ 2.85312993  2.00577715 -4.03054309]
Intercept: 0.0775987338382676
r2 score: 0.9882014752526199


After Scratch

In [3]:
class MeraRidge:
    
    def __init__(self,alpha=0.1):
        
        self.alpha = alpha
        self.coef_ = None
        self.intercept_ = None
        
    def fit(self,X_train,y_train):
        
        X_train = np.insert(X_train,0,1,axis=1)
        I = np.identity(X_train.shape[1])
        I[0][0] = 0
        result = np.linalg.inv(np.dot(X_train.T,X_train) + self.alpha * I).dot(X_train.T).dot(y_train)
        self.intercept_ = result[0]
        self.coef_ = result[1:]
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_
    

In [5]:
reg = MeraRidge()
reg.fit(x_train,y_train)
y_pred = reg.predict(x_test)
print(r2_score(y_test,y_pred))
print(reg.coef_)
print(reg.intercept_)


0.9883817901071792
[ 2.91301905  2.03005701 -4.07902468]
0.0817069094785276


Ridge Using Gradient descent

In [6]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(penalty='l2',max_iter=500,eta0=0.1,learning_rate='constant',alpha=0.001)
reg.fit(x_train,y_train)

y_pred = reg.predict(x_test)
print("R2 score",r2_score(y_test,y_pred))
print(reg.coef_)
print(reg.intercept_)

R2 score 0.985418701384346
[ 3.39751222  2.08993413 -3.83855157]
[0.36401347]


In [7]:
class MeraRidgeGD:
    
    def __init__(self,epochs,learning_rate,alpha):
        
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.alpha = alpha
        self.coef_ = None
        self.intercept_ = None
        
    def fit(self,X_train,y_train):
        
        self.coef_ = np.ones(X_train.shape[1])
        self.intercept_ = 0
        thetha = np.insert(self.coef_,0,self.intercept_)
        
        X_train = np.insert(X_train,0,1,axis=1)
        
        for i in range(self.epochs):
            thetha_der = np.dot(X_train.T,X_train).dot(thetha) - np.dot(X_train.T,y_train) + self.alpha*thetha
            thetha = thetha - self.learning_rate*thetha_der
        
        self.coef_ = thetha[1:]
        self.intercept_ = thetha[0]
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [8]:
reg = MeraRidgeGD(epochs=500,alpha=0.001,learning_rate=0.005)
reg.fit(x_train,y_train)

y_pred = reg.predict(x_test)
print("R2 score",r2_score(y_test,y_pred))
print(reg.coef_)
print(reg.intercept_)

R2 score 0.9883773291914116
[ 2.91975445  2.0327699  -4.08443022]
0.08215967308821945
